In [1]:
options(warn = -1)
knitr::opts_chunk$set(message = FALSE, warning = FALSE)  

In [47]:
library(Hmisc)
library(tidyverse)
library(tableone)
library(MASS)
library(Matching)

In [48]:
getHdata()

[1] "abm"              "acath"            "aeTestData"       "ari_other"       
 [5] "ari"              "bacteremia"       "birth.estriol"    "bodyfat"         
 [9] "boston"           "cdcaids"          "cdystonia"        "counties"        
[13] "diabetes"         "dmd"              "DominicanHTN"     "FEV"             
[17] "gbsg"             "hamdp"            "hospital"         "kprats"          
[21] "lead"             "olympics.1996"    "pbc"              "plasma"          
[25] "prostate"         "rhc"              "safety"           "schizophrenia"   
[29] "sex.age.response" "sicily"           "simhiv"           "simlongord"      
[33] "simlongord500"    "stressEcho"       "support"          "support2"        
[37] "titanic"          "titanic2"         "titanic3"         "valung"          
[41] "vlbw"             "crash2"           "esopH"            "esopH2"          
[45] "nhgh"             "gusto"            "txpo"

In [49]:
getHdata(rhc)

In [50]:
glimpse(rhc)

Rows: 5,735
Columns: 62
$ cat1     <fct> COPD, MOSF w/Sepsis, MOSF w/Malignancy, ARF, MOSF w/Sepsis, C…
$ cat2     <fct> NA, NA, MOSF w/Sepsis, NA, NA, NA, NA, Coma, NA, NA, NA, NA, …
$ ca       <fct> Yes, No, Yes, No, No, No, Metastatic, No, Yes, Yes, No, No, N…
$ sadmdte  <labelled> 11142, 11799, 12083, 11146, 12035, 12389, 12381, 11453, …
$ dschdte  <labelled> 11151, 11844, 12143, 11183, 12037, 12396, 12423, 11487, …
$ dthdte   <labelled> NA, 11844, NA, 11183, 12037, NA, NA, 11491, NA, NA, NA, …
$ lstctdte <labelled> 11382, 11844, 12400, 11182, 12036, 12590, 12616, 11490, …
$ death    <fct> No, Yes, No, Yes, Yes, No, No, Yes, No, No, No, No, Yes, Yes,…
$ cardiohx <labelled> 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,…
$ chfhx    <labelled> 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ dementhx <labelled> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,…
$ psychhx  <labelled> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ chrpulhx <labe

##
* ICU patients in 5 Hospitals
* treatment : rhc or not
* outcome : death : yes or no
* confounders : demographics, insurances etc
* 2184 treated, 3551 control  

In [51]:
#create a data set with just these variables
ARF<-as.numeric(rhc$cat1=='ARF')
CHF<-as.numeric(rhc$cat1=='CHF')
Cirr<-as.numeric(rhc$cat1=='Cirrhosis')
colcan<-as.numeric(rhc$cat1=='Colon Cancer')
Coma<-as.numeric(rhc$cat1=='Coma')
COPD<-as.numeric(rhc$cat1=='COPD')
lungcan<-as.numeric(rhc$cat1=='Lung Cancer')
MOSF<-as.numeric(rhc$cat1=='MOSF w/Malignancy')
sepsis<-as.numeric(rhc$cat1=='MOSF w/Sepsis')
female<-as.numeric(rhc$sex=='Female')
died<-as.numeric(rhc$death=='Yes')
age<-rhc$age
treatment<-as.numeric(rhc$swang1=='RHC')
meanbp1<-rhc$meanbp1


In [52]:
df <- cbind(ARF,CHF,Cirr,colcan,Coma,lungcan,MOSF,sepsis,
              age,female,meanbp1,treatment,died)
df <- data.frame(df)

In [53]:
glimpse(df)

Rows: 5,735
Columns: 13
$ ARF       <dbl> 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, …
$ CHF       <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ Cirr      <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ colcan    <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ Coma      <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ lungcan   <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ MOSF      <dbl> 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, …
$ sepsis    <dbl> 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, …
$ age       <dbl> 70.25098, 78.17896, 46.09198, 75.33197, 67.90997, 86.07794, …
$ female    <dbl> 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, …
$ meanbp1   <dbl> 41, 63, 57, 55, 65, 115, 67, 128, 53, 73, 66, 50, 53, 77, 67…
$ treatment <dbl> 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, …
$ died      <dbl

In [54]:
#covariates
xvars<-c("ARF","CHF","Cirr","colcan","Coma","lungcan","MOSF","sepsis",
         "age","female","meanbp1")

In [55]:
table1 <- CreateTableOne(vars = xvars, strata = "treatment" , data = df, test = FALSE)
# include standardized mean difference (SMD)
print(table1,smd=TRUE)

# concerned SMD > 0.1 

                     Stratified by treatment
                      0             1             SMD   
  n                    3551          2184               
  ARF (mean (SD))      0.45 (0.50)   0.42 (0.49)   0.059
  CHF (mean (SD))      0.07 (0.25)   0.10 (0.29)   0.095
  Cirr (mean (SD))     0.05 (0.22)   0.02 (0.15)   0.145
  colcan (mean (SD))   0.00 (0.04)   0.00 (0.02)   0.038
  Coma (mean (SD))     0.10 (0.29)   0.04 (0.20)   0.207
  lungcan (mean (SD))  0.01 (0.10)   0.00 (0.05)   0.095
  MOSF (mean (SD))     0.07 (0.25)   0.07 (0.26)   0.018
  sepsis (mean (SD))   0.15 (0.36)   0.32 (0.47)   0.415
  age (mean (SD))     61.76 (17.29) 60.75 (15.63)  0.061
  female (mean (SD))   0.46 (0.50)   0.41 (0.49)   0.093
  meanbp1 (mean (SD)) 84.87 (38.87) 68.20 (34.24)  0.455


In [35]:
# greedy matching on Mahalanobis distance
greedy.match <- Match(Tr = treatment, M=1, X = df[xvars], replace = FALSE)
match_data <- df[unlist(greedy.match[c("index.treated","index.control")]), ]

In [38]:
#get table 1 for matched data with standardized differences
matchedtab1<-CreateTableOne(vars=xvars, strata ="treatment", 
                            data=match_data, test = FALSE)
print(matchedtab1, smd = TRUE)

                     Stratified by treatment
                      0             1             SMD   
  n                    2184          2184               
  ARF (mean (SD))      0.42 (0.49)   0.42 (0.49)   0.006
  CHF (mean (SD))      0.10 (0.29)   0.10 (0.29)  <0.001
  Cirr (mean (SD))     0.02 (0.15)   0.02 (0.15)  <0.001
  colcan (mean (SD))   0.00 (0.02)   0.00 (0.02)  <0.001
  Coma (mean (SD))     0.04 (0.20)   0.04 (0.20)  <0.001
  lungcan (mean (SD))  0.00 (0.05)   0.00 (0.05)  <0.001
  MOSF (mean (SD))     0.07 (0.26)   0.07 (0.26)  <0.001
  sepsis (mean (SD))   0.24 (0.43)   0.32 (0.47)   0.177
  age (mean (SD))     61.53 (16.15) 60.75 (15.63)  0.049
  female (mean (SD))   0.44 (0.50)   0.41 (0.49)   0.042
  meanbp1 (mean (SD)) 73.12 (34.28) 68.20 (34.24)  0.144


In [40]:
#outcome analysis
y_trt<-match_data$died[match_data$treatment==1]
y_con<-match_data$died[match_data$treatment==0]

In [41]:

#pairwise difference
diffy<-y_trt-y_con

#paired t-test
t.test(diffy)


	One Sample t-test

data:  diffy
t = 3.9289, df = 2183, p-value = 8.799e-05
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 0.02706131 0.08099730
sample estimates:
mean of x 
0.0540293 


In [42]:

#McNemar test
table(y_trt,y_con)

mcnemar.test(matrix(c(973,513,395,303),2,2))


     y_con
y_trt   0   1
    0 303 395
    1 513 973


	McNemar's Chi-squared test with continuity correction

data:  matrix(c(973, 513, 395, 303), 2, 2)
McNemar's chi-squared = 15.076, df = 1, p-value = 0.0001033


### Propensity Score Matching 

In [58]:
#fit a propensity score model. logistic regression
psmodel <- glm(treatment ~ ARF+CHF+Cirr+colcan+Coma+lungcan+MOSF+
               sepsis+age+female+meanbp1,
   family = binomial(), data = df)


In [61]:
summary(psmodel)


Call:
glm(formula = treatment ~ ARF + CHF + Cirr + colcan + Coma + 
    lungcan + MOSF + sepsis + age + female + meanbp1, family = binomial(), 
    data = df)

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.7299670  0.1997692  -3.654 0.000258 ***
ARF          1.2931956  0.1487784   8.692  < 2e-16 ***
CHF          1.6804704  0.1715672   9.795  < 2e-16 ***
Cirr         0.5234506  0.2181458   2.400 0.016416 *  
colcan       0.0295468  1.0985361   0.027 0.978542    
Coma         0.7013451  0.1854937   3.781 0.000156 ***
lungcan     -0.0869570  0.5039331  -0.173 0.863000    
MOSF         1.3046587  0.1772705   7.360 1.84e-13 ***
sepsis       2.0433604  0.1545437  13.222  < 2e-16 ***
age         -0.0031374  0.0017289  -1.815 0.069567 .  
female      -0.1697903  0.0583574  -2.909 0.003620 ** 
meanbp1     -0.0109824  0.0008217 -13.366  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family 

In [64]:
# create propensity score
pscore <- psmodel$fitted.values


In [66]:
pscore[1:3]

1         2         3 
0.1977132 0.5514264 0.4095798

In [68]:
# It maps probabilities (ranging between 0 and 1) to values from −∞ to +∞
logit <- function(p) {
    log(p) - log(1-p)
}

logit(pscore)[1:3]

1          2          3 
-1.4006489  0.2064356 -0.3657028

In [80]:
# greedy matching on logit(PS) using Match with a caliper
logit <- function(p) {
    log(p)-log(1-p)
}
psmatch<-Match(Tr=df$treatment,M=1,X=logit(pscore),replace=FALSE,caliper=.2)
matched<-df[unlist(psmatch[c("index.treated","index.control")]), ]
xvars<-c("ARF","CHF","Cirr","colcan","Coma","lungcan","MOSF","sepsis",
         "age","female","meanbp1")

In [86]:
#get standardized differences
matchedtab1<-CreateTableOne(vars=xvars, strata ="treatment", 
                            data=matched, test = FALSE)
print(matchedtab1, smd = TRUE)

                     Stratified by treatment
                      0             1             SMD   
  n                    1932          1932               
  ARF (mean (SD))      0.47 (0.50)   0.47 (0.50)   0.009
  CHF (mean (SD))      0.10 (0.29)   0.09 (0.29)   0.004
  Cirr (mean (SD))     0.02 (0.15)   0.03 (0.16)   0.010
  colcan (mean (SD))   0.00 (0.04)   0.00 (0.02)   0.032
  Coma (mean (SD))     0.05 (0.21)   0.05 (0.22)   0.007
  lungcan (mean (SD))  0.00 (0.05)   0.00 (0.05)  <0.001
  MOSF (mean (SD))     0.08 (0.28)   0.08 (0.27)   0.008
  sepsis (mean (SD))   0.25 (0.43)   0.25 (0.43)   0.010
  age (mean (SD))     60.93 (18.00) 60.93 (15.50) <0.001
  female (mean (SD))   0.43 (0.50)   0.43 (0.49)   0.013
  meanbp1 (mean (SD)) 71.20 (33.65) 70.98 (35.02)  0.006


In [87]:

#outcome analysis
y_trt<-matched$died[matched$treatment==1]
y_con<-matched$died[matched$treatment==0]

#pairwise difference
diffy<-y_trt-y_con

#paired t-test
t.test(diffy)




	One Sample t-test

data:  diffy
t = 3.0098, df = 1931, p-value = 0.002648
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 0.01568901 0.07437310
sample estimates:
 mean of x 
0.04503106 
